# Escape Room Saturation and Opportunities

## Phoenix, AZ

In [1]:
#taking care of all the imports
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests #for getting webpage data

!conda install -c conda-forge beautifulsoup4 --yes

from bs4 import BeautifulSoup #for web-scraping

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

### Getting initial Foursquare data about Phoenix Area Escape Rooms

In [2]:
#getting the latitude/longitude for Phoenix
geo = Nominatim()
phxloc = geo.geocode('Phoenix')
phx_lat=phxloc.latitude
phx_lng=phxloc.longitude

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [3]:
#Loading Foursquare credentials and other query variables
CLIENT_ID = '0SORGAFMXO02X4VNKQPWTGGSFTPHLJPX215FAS0TSWA4XX4G' # your Foursquare ID
CLIENT_SECRET = 'FUEFJ1YFLRXK3CY2VMYILJLXYIL3OP03Z524NRYEJTFSOLSW' # your Foursquare Secret
VERSION = '20181120' # Foursquare API version
LIMIT=100

In [4]:
#Function to pull out the category id from a table entry that still has lingering JSON features
#Accepts a row as parameter
def get_id(r):
    cat_list=r['referralId']
    
    if(len(cat_list)==0):  #then there is no category
        return None
    else:
        return cat_list[4:cat_list.find('-',-10)]

In [5]:
#Function to pull out the category name from a table entry that still has lingering JSON features
#Accepts a row as parameter
def get_category_type(r):
    cat_list=r['categories']
    
    if(len(cat_list)==0):  #then there is no category
        return None
    else:
        return cat_list[0]['name']

In [6]:
#getting the url for a search for venues with the word 'escape'
radius=100000
search_string='escape'
url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}".format(CLIENT_ID,
                                                                                                                                  CLIENT_SECRET,
                                                                                                                                  VERSION,
                                                                                                                                  phx_lat,phx_lng,
                                                                                                                                  radius,LIMIT,
                                                                                                                                  search_string)
results=requests.get(url).json()
results
er_df_json=results['response']['groups'][0]['items']
er_df=json_normalize(er_df_json)
er_df2=er_df[['referralId','venue.categories','venue.location.city','venue.location.lat','venue.location.lng',
             'venue.location.formattedAddress','venue.location.postalCode','venue.name']]
er_df2.columns=[col.split('.')[-1] for col in er_df2.columns]

er_df2.insert(0,'id',er_df2.apply(get_id,axis=1))
#filter down to the category name
er_df2['categories']=er_df2.apply(get_category_type,axis=1)
print(er_df.shape)
er_df2.head()

(52, 28)


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,referralId,categories,city,lat,lng,formattedAddress,postalCode,name
0,54ac0e75498e48c9dee98ffd,e-0-54ac0e75498e48c9dee98ffd-0,General Entertainment,Scottsdale,33.492954,-111.929806,"[7017 E Main St (at N Goldwater Blvd), Scottsd...",85251,Escape The Room
1,500e1d11e4b04e4dcc609f7a,e-0-500e1d11e4b04e4dcc609f7a-1,College Classroom,Tempe,33.417919,-111.911057,"[Tempe, AZ 85281, United States]",85281,Escape
2,4f32958519836c91c7e60c33,e-0-4f32958519836c91c7e60c33-2,Food,Phoenix,33.448265,-112.071276,"[201 E Washington St, Phoenix, AZ, United States]",NaN,Steak Escape
3,4b4a8855f964a5201c8a26e3,e-0-4b4a8855f964a5201c8a26e3-3,Garden,Phoenix,33.460940,-112.076794,"[1125 N 3rd St (N of Roosevelt St), Phoenix, A...",85004,Japanese Friendship Garden
4,4dbdcb305da3ff58ec6c24b9,e-0-4dbdcb305da3ff58ec6c24b9-4,Lounge,Phoenix,33.445595,-112.066782,"[401 E Jefferson St (at Chase Field), Phoenix,...",85004,Audi Quattro Lounge


In [7]:
#filter venue list to just the escape rooms based off of what category they're in
er_filter=er_df2[(er_df2['categories']=='Theme Park') | (er_df2['categories']=='General Entertainment') 
                       | (er_df2['categories']=='Theme Park Ride / Attraction') | (er_df2['categories']=='Recreation Center') 
                       | (er_df2['categories']=='Arts & Entertainment')]
#filtering out unnecessary zip codes using a 'minimum' zip code for the area
er_filter=er_filter[er_filter['postalCode']>'80000']
er_filter.reset_index(inplace=True)
er_filter.drop(['index','referralId'],axis=1,inplace=True)
er_filter.head()

,id,categories,city,lat,lng,formattedAddress,postalCode,name
0,54ac0e75498e48c9dee98ffd,General Entertainment,Scottsdale,33.492954,-111.929806,"[7017 E Main St (at N Goldwater Blvd), Scottsd...",85251,Escape The Room
1,579a86e1498e7d496bb0e998,Theme Park,Phoenix,33.432011,-112.065516,"[1638 S 7th St, Phoenix, AZ 85034, United States]",85034,PanIQ Escape Room Phoenix
2,5778734ecd10013c3cf36556,Recreation Center,Tempe,33.426061,-111.940440,"[414 South Mill Avenue #112, Tempe, AZ 85281, ...",85281,Escape Zone
3,5ade261e4acb196e97a908f8,General Entertainment,Tempe,33.396057,-111.926678,"[3030 South Rural Road #108, Tempe, AZ 85282, ...",85282,Escape Narrative
4,575e0b0a498e5384597beb29,General Entertainment,Phoenix,33.525130,-112.065735,"[6040 N 7th St, Phoenix, AZ 85014, United States]",85014,Phoenix Escape Room


In [8]:
#creating dataframe to hold the rating info as well
er_ratings=pd.DataFrame(er_filter)
er_ratings.drop('name',axis=1,inplace=True)
#adding rating-related categories
er_ratings.insert(1,'name',er_filter['name'])
er_ratings.insert(2,'likes','')
er_ratings.insert(3,'dislikes','')
#converting zip codes to integers
er_ratings['postalCode']=er_ratings['postalCode'].astype(int)
er_ratings.head()

,id,name,likes,dislikes,categories,city,lat,lng,formattedAddress,postalCode
0,54ac0e75498e48c9dee98ffd,Escape The Room,,,General Entertainment,Scottsdale,33.492954,-111.929806,"[7017 E Main St (at N Goldwater Blvd), Scottsd...",85251
1,579a86e1498e7d496bb0e998,PanIQ Escape Room Phoenix,,,Theme Park,Phoenix,33.432011,-112.065516,"[1638 S 7th St, Phoenix, AZ 85034, United States]",85034
2,5778734ecd10013c3cf36556,Escape Zone,,,Recreation Center,Tempe,33.426061,-111.940440,"[414 South Mill Avenue #112, Tempe, AZ 85281, ...",85281
3,5ade261e4acb196e97a908f8,Escape Narrative,,,General Entertainment,Tempe,33.396057,-111.926678,"[3030 South Rural Road #108, Tempe, AZ 85282, ...",85282
4,575e0b0a498e5384597beb29,Phoenix Escape Room,,,General Entertainment,Phoenix,33.525130,-112.065735,"[6040 N 7th St, Phoenix, AZ 85014, United States]",85014


In [9]:
#getting Escape Room Rating Data 
for i in range(0,er_ratings.shape[0]):
    #formulating the query url
    url2="https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}".format(er_filter.loc[i]['id'],CLIENT_ID,CLIENT_SECRET,VERSION)
    response2=requests.get(url2).json()
    if response2['response']['venue']['likes']!='false':  # checks if there is a likes count
        er_ratings.loc[i,'likes']=int(response2['response']['venue']['likes']['count'])
    else:
        er_ratings.loc[i,'likes']=0

    if str(response2['response']['venue']['dislike'])!='False': # checks if there is a dislike count

        er_ratings.loc[i,'dislikes']=response2['response']['venue']['dislike']['count']
    else:
        er_ratings.loc[i,'dislikes']=0
er_ratings.head()

,id,name,likes,dislikes,categories,city,lat,lng,formattedAddress,postalCode
0,54ac0e75498e48c9dee98ffd,Escape The Room,32,0,General Entertainment,Scottsdale,33.492954,-111.929806,"[7017 E Main St (at N Goldwater Blvd), Scottsd...",85251
1,579a86e1498e7d496bb0e998,PanIQ Escape Room Phoenix,1,0,Theme Park,Phoenix,33.432011,-112.065516,"[1638 S 7th St, Phoenix, AZ 85034, United States]",85034
2,5778734ecd10013c3cf36556,Escape Zone,1,0,Recreation Center,Tempe,33.426061,-111.940440,"[414 South Mill Avenue #112, Tempe, AZ 85281, ...",85281
3,5ade261e4acb196e97a908f8,Escape Narrative,0,0,General Entertainment,Tempe,33.396057,-111.926678,"[3030 South Rural Road #108, Tempe, AZ 85282, ...",85282
4,575e0b0a498e5384597beb29,Phoenix Escape Room,1,0,General Entertainment,Phoenix,33.525130,-112.065735,"[6040 N 7th St, Phoenix, AZ 85014, United States]",85014


In [10]:
#adding in population density data
csv_address='https://s3.amazonaws.com/SplitwiseBlogJB/Zipcode-ZCTA-Population-Density-And-Area-Unsorted.csv'
density=pd.read_csv(csv_address)
density.head()

,Zip/ZCTA,2010 Population,Land-Sq-Mi,Density Per Sq Mile
0,601,0,64.348,0.0
1,602,0,30.613,0.0
2,603,0,31.616,0.0
3,606,0,42.309,0.0
4,610,0,35.916,0.0


In [11]:
#joining escape room data with population density info
er_final=pd.merge(er_ratings,density,how='inner',left_on='postalCode',right_on='Zip/ZCTA')
er_final.drop('Zip/ZCTA',axis=1,inplace=True)
#selecting the lowest population density of any zip code that contains an escape room
min_density=er_final["Density Per Sq Mile"].min()
print('Minimum population density to analyze: ',min_density)
er_final.head()

Minimum population density to analyze:  488.70600599999995


,id,name,likes,dislikes,categories,city,lat,lng,formattedAddress,postalCode,2010 Population,Land-Sq-Mi,Density Per Sq Mile
0,54ac0e75498e48c9dee98ffd,Escape The Room,32,0,General Entertainment,Scottsdale,33.492954,-111.929806,"[7017 E Main St (at N Goldwater Blvd), Scottsd...",85251,34390,7.369,4666.847605
1,579a86e1498e7d496bb0e998,PanIQ Escape Room Phoenix,1,0,Theme Park,Phoenix,33.432011,-112.065516,"[1638 S 7th St, Phoenix, AZ 85034, United States]",85034,5582,11.422,488.706006
2,5778734ecd10013c3cf36556,Escape Zone,1,0,Recreation Center,Tempe,33.426061,-111.940440,"[414 South Mill Avenue #112, Tempe, AZ 85281, ...",85281,57348,13.355,4294.122052
3,5b8b78990ff4f900395ffecf,Inferno Escape Room,0,0,General Entertainment,Tempe,33.421269,-111.915547,[1415 E University Dr Ste 105 (University McCl...,85281,57348,13.355,4294.122052
4,5a886d264420d87ec8a7c892,Mindspark Escape Games Tempe,0,0,Theme Park,Tempe,33.421416,-111.965606,"[1625 W University Dr, Tempe, AZ 85281, United...",85281,57348,13.355,4294.122052


### Getting Zip Code Data

In [12]:
#web page from bestplaces.net that has the desired list of zip codes
z_url='https://www.bestplaces.net/find/zip.aspx?st=az&msa=38060'
html_text=requests.get(z_url)
#creating BeautifulSoup object with parsed webpage
phx_soup=BeautifulSoup(html_text.text,'html.parser')

In [13]:
#creating new dataframe to hold zip codes
phx_raw=pd.DataFrame()

#looking for row tags
for tag in phx_soup.find_all('div'):
    if 'class' in tag.attrs:
        if tag['class']==['col-md-4']: #then it's the right type of table entry
        
    #iterate through children within the table row
            for ch in tag.children:
                if(ch.string!=None):
                    if  ch.string!='\n':
                        loc_string=ch.string[0:5]+", Arizona"
                        loc = geo.geocode(loc_string)
                        lat_temp=loc.latitude
                        lng_temp=loc.longitude
                        phx_raw=phx_raw.append([[ch.string[0:5],ch.string[7:-1],lat_temp,lng_temp]])
            
    
#add in column headers
phx_raw.columns=('PostalCode','City','Latitude','Longitude')
phx_raw.reset_index(inplace=True)
phx_raw.drop('index',axis=1,inplace=True)
phx_raw.head()

,PostalCode,City,Latitude,Longitude
0,85003,Phoenix,33.448608,-112.096624
1,85004,Phoenix,33.429208,-112.068479
2,85006,Phoenix,33.464916,-112.045621
3,85007,Phoenix,33.455628,-112.089051
4,85008,Phoenix,33.440771,-112.186425


In [14]:
#adding population density data
phx_raw['PostalCode']=phx_raw['PostalCode'].astype(int)

phx_density=pd.merge(phx_raw,density,how='inner',left_on='PostalCode',right_on='Zip/ZCTA')
phx_density.drop('Zip/ZCTA',axis=1,inplace=True)

#filtering out zip codes with lower population densities and resetting the index
phx_density=phx_density[phx_density["Density Per Sq Mile"]>=min_density]
phx_density.reset_index(inplace=True)
phx_density.drop('index',axis=1,inplace=True)
phx_density.head()

,PostalCode,City,Latitude,Longitude,2010 Population,Land-Sq-Mi,Density Per Sq Mile
0,85003,Phoenix,33.448608,-112.096624,9369,1.851,5061.588331
1,85004,Phoenix,33.429208,-112.068479,4965,2.040,2433.823529
2,85006,Phoenix,33.464916,-112.045621,25742,3.904,6593.750000
3,85007,Phoenix,33.455628,-112.089051,14040,4.515,3109.634551
4,85008,Phoenix,33.440771,-112.186425,56145,10.467,5364.001146


In [15]:
radius=500 
#creating a new dataframe to hold all of venue data
phx_venue_list=pd.DataFrame()

#cycle through the rows of postal codes
for i in range(0,phx_density.shape[0]):
    #querying foursquare for venue info surrounding the postal code
    vurl="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,
                                                                                                                              CLIENT_SECRET,
                                                                                                                              VERSION,
                                                                                                                              phx_density.loc[i]['Latitude'],
                                                                                                                              phx_density.loc[i]['Longitude'],
                                                                                                                              radius,LIMIT)
    results3=requests.get(vurl).json()
    
    #converting the appropriate parts of the json file into a dataframe
    venue_list_json=results3['response']['groups'][0]['items']
    venue_list=json_normalize(venue_list_json)

    if venue_list.shape[0]!=0:  #checking to make sure there's data in the venue list for this postal code
        #pull the needed columns and adjust titles
        venue_list=venue_list.loc[:,['venue.name','venue.categories','venue.location.lat','venue.location.lng']]
        venue_list.columns=[col.split('.')[-1] for col in venue_list.columns]
        #filter down to the category name
        venue_list['categories']=venue_list.apply(get_category_type,axis=1)
        
        #cycle through the venues and place the information in the final dataframe
        for j in range(0,venue_list.shape[0]):
            phx_venue_list=phx_venue_list.append([(phx_density.loc[i]['PostalCode'],phx_density.loc[i]['City'],phx_density.loc[i]['Latitude'],
                                                   phx_density.loc[i]['Longitude'],venue_list.loc[j]['name'],venue_list.loc[j]['lat'],
                                                   venue_list.loc[j]['lng'],venue_list.loc[j]['categories'])])
    else:  #there's no venue data for this postal code--simply put the postal code info in the main dataframe
        phx_venue_list=phx_venue_list.append([(phx_density.loc[i]['PostalCode'],phx_density.loc[i]['City'],phx_density.loc[i]['Latitude'],
                                               phx_density.loc[i]['Longitude'],'NA','NA','NA','NA')])

#adjust column names
phx_venue_list.columns=('PostalCode','City','Latitude','Longitude','Venue','Venue_Latitude','Venue_Longitude','Venue_Category')

phx_venue_list.head()


,PostalCode,City,Latitude,Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,85003,Phoenix,33.448608,-112.096624,Wesley Bolin Memorial Plaza,33.4481,-112.095,Plaza
0,85003,Phoenix,33.448608,-112.096624,Arizona Capitol Museum,33.4482,-112.097,History Museum
0,85003,Phoenix,33.448608,-112.096624,Arizona State Capitol,33.4482,-112.096,Capitol Building
0,85003,Phoenix,33.448608,-112.096624,La Canasta Capitolio,33.4509,-112.097,Mexican Restaurant
0,85003,Phoenix,33.448608,-112.096624,Food Truck Friday @ AZ State Capitol Lawn,33.4468,-112.094,Food Truck


In [16]:
phx_venue_list.shape

(981, 8)

### Processing PHX Venue Information

In [17]:
#creating one-hot encoding of venues in each postal code
phx_onehot=pd.get_dummies(phx_venue_list['Venue_Category'])
#putting in the postal code column 
phx_onehot.insert(0,'PostalCode',phx_venue_list['PostalCode'])

#creating a mean of each type of venue per postal code
phx_group=phx_onehot.groupby('PostalCode').mean().reset_index()
phx_group.head()

,PostalCode,ATM,Advertising Agency,Airport,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Argentinian Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,85003,0.0,0.0,0.0,0.0,0.105263,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,85004,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,85006,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,85007,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.214286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,85008,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Clustering on the PHX Data

In [18]:
n_clusters=20
#initiating K-Means clustering
phx_KMeans=KMeans(init='k-means++',n_init=12,n_clusters=n_clusters)
phx_KMeans.fit(phx_group.drop('PostalCode',axis=1))
phx_clust_labels=phx_KMeans.labels_
phx_clust_labels

array([ 0,  2,  2,  2,  1, 18,  2, 10, 10, 10,  2, 10, 10, 10, 19, 10, 12,
       10,  2,  2,  2, 16, 10, 10,  3,  2, 10,  1,  0,  3,  0,  9,  2,  2,
       10, 10, 10,  1,  0, 14,  0, 10, 10,  4, 10,  1, 10, 10,  2, 10,  5,
       11,  4, 17, 10,  2, 10, 15,  4, 10,  3, 10,  0, 17,  1, 19, 10,  1,
        3,  7, 10, 10, 10,  2,  1,  0,  2,  0, 10,  3, 10, 10, 13, 10,  2,
        7, 10,  0, 10,  2, 10,  1, 10, 10,  6, 10,  1,  2,  2, 19,  3,  8,
        2, 10,  3,  2,  2,  2,  5,  0], dtype=int32)

In [19]:
phx_final=phx_density
#adding cluster information
phx_final['Labels']=list(phx_clust_labels)
phx_final.head()

,PostalCode,City,Latitude,Longitude,2010 Population,Land-Sq-Mi,Density Per Sq Mile,Labels
0,85003,Phoenix,33.448608,-112.096624,9369,1.851,5061.588331,0
1,85004,Phoenix,33.429208,-112.068479,4965,2.040,2433.823529,2
2,85006,Phoenix,33.464916,-112.045621,25742,3.904,6593.750000,2
3,85007,Phoenix,33.455628,-112.089051,14040,4.515,3109.634551,2
4,85008,Phoenix,33.440771,-112.186425,56145,10.467,5364.001146,1


In [20]:
#mapping the neighborhoods
phx_map=folium.Map(location=[phx_lat,phx_lng],zoom_start=10)
colors=['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#8dd3c7','#ffffb3','#bebada',
        '#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd']
#adding postalcode markers for the clustered neighorhoods
phx_neigh=folium.map.FeatureGroup()
for pc,lat,long,label in zip(phx_final['PostalCode'],phx_final['Latitude'],phx_final['Longitude'],phx_final['Labels']):
    phx_neigh.add_child(folium.features.CircleMarker([lat,long],color=colors[label],fill_color=colors[label],fill=True,popup=str(pc),fill_opacity=0.6))

#adding popup markers for the escape rooms
for pc,lat,long,label in zip(er_final['postalCode'],er_final['lat'],er_final['lng'],er_final['name']):
    folium.Marker(location=[lat,long],popup=(label+', '+str(pc))).add_to(phx_map)
phx_map.add_child(phx_neigh)
phx_map

### PHX Escape Room Analysis

In [21]:
#sorting by the most likes
er_ratings_top=er_final.sort_values(by=['likes'],ascending=False).reset_index().drop('index',axis=1)
#joining escape room data with cluster information
er_ratings_top=pd.merge(er_ratings_top,phx_final,how='inner',left_on='postalCode',right_on='PostalCode')
er_ratings_top.drop(['id','PostalCode','City','Latitude','Longitude',"2010 Population_y","Land-Sq-Mi_y","Density Per Sq Mile_y"],axis=1,inplace=True)
er_ratings_top.head()

,name,likes,dislikes,categories,city,lat,lng,formattedAddress,postalCode,2010 Population_x,Land-Sq-Mi_x,Density Per Sq Mile_x,Labels
0,Escape The Room,32,0,General Entertainment,Scottsdale,33.492954,-111.929806,"[7017 E Main St (at N Goldwater Blvd), Scottsd...",85251,34390,7.369,4666.847605,10
1,Riddle Escape Room,5,0,General Entertainment,Scottsdale,33.474044,-111.927523,"[2334 N Scottsdale Rd Ste A120, Scottsdale, AZ...",85257,26755,6.558,4079.749924,7
2,Baffled Escape Rooms,2,0,Arts & Entertainment,Peoria,33.596260,-112.219760,"[7420 W Cactus Rd Ste B12, Peoria, AZ 85381, U...",85381,24249,6.642,3650.858175,2
3,PanIQ Escape Room Phoenix,1,0,Theme Park,Phoenix,33.432011,-112.065516,"[1638 S 7th St, Phoenix, AZ 85034, United States]",85034,5582,11.422,488.706006,2
4,Escape Zone,1,0,Recreation Center,Tempe,33.426061,-111.940440,"[414 South Mill Avenue #112, Tempe, AZ 85281, ...",85281,57348,13.355,4294.122052,0


In [22]:
er_final.describe()

,likes,dislikes,lat,lng,postalCode,2010 Population,Land-Sq-Mi,Density Per Sq Mile
count,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,2.687500,0.0,33.458335,-111.953831,85211.500000,34585.187500,9.317812,3827.296266
std,7.922279,0.0,0.101294,0.115169,111.158146,17038.633628,2.890582,1552.072330
min,0.000000,0.0,33.321269,-112.219760,85014.000000,5384.000000,4.180000,488.706006
25%,0.000000,0.0,33.409598,-111.990583,85164.250000,24648.750000,7.187250,3550.698834
50%,0.000000,0.0,33.424342,-111.931047,85254.000000,35594.500000,9.325000,4294.122052
75%,1.000000,0.0,33.500998,-111.904398,85281.250000,46736.750000,11.347000,4735.846159
max,32.000000,0.0,33.656735,-111.790055,85381.000000,57348.000000,13.355000,5928.708134


In [23]:
#getting just the zip codes that correspond to the clusters the top 2 escape rooms belong to
er_ratings_top2=phx_final[(phx_final['Labels']==er_ratings_top.loc[0,'Labels']) | (phx_final['Labels']==er_ratings_top.loc[1,'Labels'])]
print(er_ratings_top2.shape)
er_ratings_top2.head()

(41, 8)


,PostalCode,City,Latitude,Longitude,2010 Population,Land-Sq-Mi,Density Per Sq Mile,Labels
7,85013,Phoenix,33.502638,-112.082109,19314,3.718,5194.728349,10
8,85014,Phoenix,33.507543,-112.060829,24782,4.180,5928.708134,10
9,85015,Phoenix,33.499049,-112.099484,37644,4.883,7709.195167,10
11,85017,Phoenix,33.522579,-112.129560,38872,5.242,7415.490271,10
12,85018,Phoenix,33.506209,-111.995319,36065,11.114,3245.006298,10


In [24]:
#adding count of how many escape rooms are in each zip code
phx_top_picks=pd.merge(er_ratings_top2,pd.DataFrame(er_final.groupby('postalCode').count()['id']),how='left',left_on='PostalCode',right_on='postalCode')
#changing the NaN's to 0's
phx_top_picks['id']=np.nan_to_num(phx_top_picks['id'])
phx_top_picks.rename(columns={'id':"Number of Escape Rooms"},inplace=True)
phx_top_picks

,PostalCode,City,Latitude,Longitude,2010 Population,Land-Sq-Mi,Density Per Sq Mile,Labels,Number of Escape Rooms
0,85013,Phoenix,33.502638,-112.082109,19314,3.718,5194.728349,10,0.0
1,85014,Phoenix,33.507543,-112.060829,24782,4.180,5928.708134,10,1.0
2,85015,Phoenix,33.499049,-112.099484,37644,4.883,7709.195167,10,0.0
3,85017,Phoenix,33.522579,-112.129560,38872,5.242,7415.490271,10,0.0
4,85018,Phoenix,33.506209,-111.995319,36065,11.114,3245.006298,10,0.0
5,85019,Phoenix,33.494507,-112.137913,25042,3.750,6677.866667,10,0.0
6,85021,Phoenix,33.555081,-112.101109,37743,6.726,5611.507583,10,0.0
7,85023,Phoenix,33.564801,-112.077016,31274,7.308,4279.419814,10,0.0
8,85031,Phoenix,33.499870,-112.171244,30493,4.054,7521.706956,10,0.0
9,85032,Phoenix,33.617391,-112.006387,65485,12.421,5272.119797,10,0.0


####  The above table represents a list of potential zip codes that match the neighborhood profile of currently successful escape rooms, and the existing # of escape rooms in those zip codes.

### Creating additional functions to systematize this

In [25]:
#function to get latitude/longitude data
#accepts a string location and returns the latitude and longitude as a list
def latLong(loc):
    geospace = Nominatim()
    location = geospace.geocode(loc)
    lat=location.latitude
    lng=location.longitude
    return (lat,lng)
    

In [38]:
#function to produce a dataframe of escape rooms in the requested area, and to print out the recommended minimum population density to use
#accepts as parameters:  string address of a zip code csv file, Foursquare sign-in credentials, desired Foursquare version,
#                        latitude/longitude to center the search around, search radius, search limit, 
#                        and minimum acceptable zip code (as a string) 
def getEscapeRooms(zc_csv,fs_id,fs_secret,fs_version,lat,lng,radius,search_limit,zc_min):
    #getting the url for a search for venues with the word 'escape'
    search_string='escape'
    fs_url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}".format(fs_id,
                                                                                                                                  fs_secret,
                                                                                                                                  fs_version,
                                                                                                                                  lat,lng,
                                                                                                                                  radius,search_limit,
                                                                                                                                  search_string)
    search_results=requests.get(fs_url).json()
    search_json=search_results['response']['groups'][0]['items']
    results_df=json_normalize(search_json)
    results_df=results_df[['referralId','venue.categories','venue.location.city','venue.location.lat','venue.location.lng',
                           'venue.location.formattedAddress','venue.location.postalCode','venue.name']]
    results_df.columns=[col.split('.')[-1] for col in results_df.columns]
    results_df.insert(0,'id',results_df.apply(get_id,axis=1))
    #filter down to the category name
    results_df['categories']=results_df.apply(get_category_type,axis=1)
    #filtering out the venues that aren't escape rooms, based off of their categories
    filtered_df=results_df[(results_df['categories']=='Theme Park') | (results_df['categories']=='General Entertainment') 
                       | (results_df['categories']=='Theme Park Ride / Attraction') | (results_df['categories']=='Recreation Center') 
                       | (results_df['categories']=='Arts & Entertainment')]
    filtered_df=filtered_df[filtered_df['postalCode']>zc_min]
    filtered_df.reset_index(inplace=True)
    filtered_df.drop(['index','referralId'],axis=1,inplace=True)
    
    #creating dataframe to hold the rating info as well
    er_ratings_df=pd.DataFrame(filtered_df)
    er_ratings_df.drop('name',axis=1,inplace=True)
    #adding rating-related categories
    er_ratings_df.insert(1,'name',filtered_df['name'])
    er_ratings_df.insert(2,'likes','')
    er_ratings_df.insert(3,'dislikes','')
    #converting zip codes to integers
    er_ratings_df['postalCode']=er_ratings_df['postalCode'].astype(int)
    
    #getting Escape Room Rating Data 
    for i in range(0,er_ratings_df.shape[0]):
        #formulating the query url
        fs_url2="https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}".format(filtered_df.loc[i]['id'],fs_id,
                                                                                                    fs_secret,fs_version)
        search_results=requests.get(fs_url2).json()
        if search_results['response']['venue']['likes']!='false':  # checks if there is a likes count
            er_ratings_df.loc[i,'likes']=int(search_results['response']['venue']['likes']['count'])
        else:
            er_ratings_df.loc[i,'likes']=0

        if str(search_results['response']['venue']['dislike'])!='False': # checks if there is a dislike count
            er_ratings_df.loc[i,'dislikes']=search_results['response']['venue']['dislike']['count']
        else:
            er_ratings_df.loc[i,'dislikes']=0
    
    #adding in population density data
    pop_density=pd.read_csv(zc_csv)
    #joining escape room data with population density info
    er_final_df=pd.merge(er_ratings_df,pop_density,how='inner',left_on='postalCode',right_on='Zip/ZCTA')
    er_final_df.drop('Zip/ZCTA',axis=1,inplace=True)
    
    #status check
    print("Escape Room Data Gathering Complete")
    #selecting the lowest population density of any zip code that contains an escape room
    print('**************Recommended minimum population density to analyze: ',er_final_df["Density Per Sq Mile"].min(),'*****************')
    return er_final_df

In [27]:
#Function to return a dataframe with zip code data that has been split into a selected number of clusters using K-Means clustering
#Accepts as parameters:  url of csv with population density data, url of zip code data from BestPlaces.net for the desired area, # of clusters, 
#                        minimum population density of zip codes to be considered,Foursquare sign-in credentials, desired Foursquare version, 
#                        search limit, search radius, and the state in which the zip codes reside (as a string)
def getZCData(zc_csv,zc_url,num_clusters,minimum_density,fs_id,fs_secret,fs_version,search_limit,radius,state):
    #web page from bestplaces.net that has the desired list of zip codes
    zc_html=requests.get(zc_url)
    #creating BeautifulSoup object with parsed webpage
    zc_soup=BeautifulSoup(zc_html.text,'html.parser')
    
    #creating new dataframe to hold zip codes
    zc_raw=pd.DataFrame()
    zc_geo=Nominatim()
    #looking for row tags
    for tag in zc_soup.find_all('div'):
        if 'class' in tag.attrs:
            if tag['class']==['col-md-4']: #then it's the right type of table entry
                #iterate through children within the table row
                for ch in tag.children:
                    if(ch.string!=None):
                        if  ch.string!='\n':
                            loc_string=ch.string[0:5]+", "+state
                            loc = latLong(loc_string)
                            zc_raw=zc_raw.append([[ch.string[0:5],ch.string[7:-1],loc[0],loc[1]]])
            
    
    #add in column headers and reset index
    zc_raw.columns=('PostalCode','City','Latitude','Longitude')
    zc_raw.reset_index(inplace=True)
    zc_raw.drop('index',axis=1,inplace=True)
    
    #adding population density data
    zc_raw['PostalCode']=zc_raw['PostalCode'].astype(int)
    pop_density=pd.read_csv(zc_csv)
    #merging zip code data and population density data
    zc_density=pd.merge(zc_raw,pop_density,how='inner',left_on='PostalCode',right_on='Zip/ZCTA')
    zc_density.drop('Zip/ZCTA',axis=1,inplace=True)

    #filtering out zip codes with lower population densities and resetting the index
    zc_density=zc_density[zc_density["Density Per Sq Mile"]>=minimum_density]
    zc_density.reset_index(inplace=True)
    zc_density.drop('index',axis=1,inplace=True)
    
    #creating a new dataframe to hold all of venue data
    zc_venue_list=pd.DataFrame()

    #cycle through the rows of postal codes
    for i in range(0,zc_density.shape[0]):
        #querying foursquare for venue info surrounding the postal code
        fs_url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(fs_id,
                                                                                                                              fs_secret,
                                                                                                                              fs_version,
                                                                                                                              zc_density.loc[i]['Latitude'],
                                                                                                                              zc_density.loc[i]['Longitude'],
                                                                                                                              radius,search_limit)
        search_results=requests.get(fs_url).json()
    
        #converting the appropriate parts of the json file into a dataframe
        search_json=search_results['response']['groups'][0]['items']
        results_df=json_normalize(search_json)

        if results_df.shape[0]!=0:  #checking to make sure there's data in the venue list for this postal code
            #pull the needed columns and adjust titles
            results_df=results_df.loc[:,['venue.name','venue.categories','venue.location.lat','venue.location.lng']]
            results_df.columns=[col.split('.')[-1] for col in results_df.columns]
            #filter down to the category name
            results_df['categories']=results_df.apply(get_category_type,axis=1)
        
            #cycle through the venues and place the information in the final dataframe
            for j in range(0,results_df.shape[0]):
                zc_venue_list=zc_venue_list.append([(zc_density.loc[i]['PostalCode'],zc_density.loc[i]['City'],zc_density.loc[i]['Latitude'],
                                                 zc_density.loc[i]['Longitude'],results_df.loc[j]['name'],results_df.loc[j]['lat'],
                                                 results_df.loc[j]['lng'],results_df.loc[j]['categories'])])
        else:  #there's no venue data for this postal code--simply put the postal code info in the main dataframe
            zc_venue_list=zc_venue_list.append([(zc_density.loc[i]['PostalCode'],zc_density.loc[i]['City'],zc_density.loc[i]['Latitude'],
                                                 zc_density.loc[i]['Longitude'],'NA','NA','NA','NA')])

    #adjust column names
    zc_venue_list.columns=('PostalCode','City','Latitude','Longitude','Venue','Venue_Latitude','Venue_Longitude','Venue_Category')

    #status check
    print("Venues Processed")
    
    #creating one-hot encoding of venues in each postal code
    zc_onehot=pd.get_dummies(zc_venue_list['Venue_Category'])
    #putting in the postal code column 
    zc_onehot.insert(0,'PostalCode',zc_venue_list['PostalCode'])

    #creating a mean of each type of venue per postal code
    zc_group=zc_onehot.groupby('PostalCode').mean().reset_index()
    
    #initiating K-Means clustering
    zc_KMeans=KMeans(init='k-means++',n_init=12,n_clusters=num_clusters)
    zc_KMeans.fit(zc_group.drop('PostalCode',axis=1))
    
    #adding cluster information
    zc_density['Labels']=list(zc_KMeans.labels_)
    
    #status check
    print("Zip Code Analysis Complete")
    return zc_density

In [28]:
#Function to map zip code clustering and escape room locations
#Accepts as parameters:  dataframe of escape room data, dataframe of zip code data, and latitude/longitude data to base the map on
def mapEscapeRooms(er_df,zc_df,main_lat,main_lng):
    
    #mapping the neighborhoods
    zc_map=folium.Map(location=[main_lat,main_lng],zoom_start=10)
    colors=['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#8dd3c7','#ffffb3','#bebada',
            '#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd']
    #adding postalcode markers for the clustered neighorhoods
    neigh=folium.map.FeatureGroup()
    for pc,lat,long,label in zip(zc_df['PostalCode'],zc_df['Latitude'],zc_df['Longitude'],zc_df['Labels']):
        neigh.add_child(folium.features.CircleMarker([lat,long],color=colors[label],fill_color=colors[label],fill=True,popup=str(pc),fill_opacity=0.6))

    #adding popup markers for the escape rooms
    for pc,lat,long,label in zip(er_df['postalCode'],er_df['lat'],er_df['lng'],er_df['name']):
        folium.Marker(location=[lat,long],popup=(label+', '+str(pc))).add_to(zc_map)
    zc_map.add_child(neigh)
    return zc_map

In [59]:
#Function to produce a dataframe of recommended zip codes for escape rooms, based off of which existing escape rooms are most popular,
#and how many escape rooms are currently in each zip code
#Accepts as parameters:  dataframe of escape room data, dataframe of zip code data
def analyzeEscapeRooms(er_df,zc_df):
    #sorting by the most likes
    sorted_er_df=er_df.sort_values(by=['likes'],ascending=False).reset_index().drop('index',axis=1)
    #joining escape room data with cluster information
    sorted_er_df=pd.merge(sorted_er_df,zc_df,how='inner',left_on='postalCode',right_on='PostalCode')
    sorted_er_df.drop(['id','PostalCode','City','Latitude','Longitude',"2010 Population_y","Land-Sq-Mi_y","Density Per Sq Mile_y"],axis=1,inplace=True)
    #getting just the zip codes that correspond to the clusters the top 2 escape rooms belong to
    top2=zc_df[(zc_df['Labels']==sorted_er_df.loc[0,'Labels']) | (zc_df['Labels']==sorted_er_df.loc[1,'Labels'])]
    #adding count of how many escape rooms are in each zip code
    top_picks=pd.merge(top2,pd.DataFrame(er_df.groupby('postalCode').count()['id']),how='left',left_on='PostalCode',right_on='postalCode')
    #changing the NaN's to 0's
    top_picks['id']=np.nan_to_num(top_picks['id'])
    top_picks.rename(columns={'id':"Number of Escape Rooms"},inplace=True)
    return top_picks

### Salt Lake City Analysis

In [30]:
slc_data=latLong('Salt Lake City, UT')
slc_er=getEscapeRooms(csv_address,CLIENT_ID,CLIENT_SECRET,VERSION,slc_data[0],slc_data[1],100000,LIMIT,'8000')
slc_er

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Escape Room Data Gathering Complete
**************Recommended minimum population density to analyze:  1578.853331 *****************


,id,name,likes,dislikes,categories,city,lat,lng,formattedAddress,postalCode,2010 Population,Land-Sq-Mi,Density Per Sq Mile
0,54ee07d1498ed385b9b7b75c,The Escape Key,1,0,General Entertainment,Salt Lake City,40.760907,-111.880704,"[353 E 400 S, Salt Lake City, UT 84111, United...",84111,10744,1.434,7492.329149
1,5572455f498eef85ac0f3329,Mystery Escape Room,3,0,General Entertainment,Salt Lake City,40.765775,-111.903416,"[130 S Rio Grande St, Salt Lake City, UT 84101...",84101,5277,1.819,2901.044530
2,57e5864d498e508a7852134d,Alcatraz Escape Games,4,0,General Entertainment,Draper,40.521074,-111.892630,"[12674 S Pony Express Rd Ste 1, Draper, UT 840...",84020,39765,25.186,1578.853331
3,58a554e15cab2f5be9659939,Hidden Chambers! Escape Room Adventures,0,0,General Entertainment,Sandy,40.596489,-111.853572,"[8522 S 1300 E Ste 202, Sandy, UT 84094, Unite...",84094,28069,5.106,5497.258128


In [49]:
slc_zc=getZCData(csv_address,'https://www.bestplaces.net/find/zip.aspx?msa=41620&st=Utah',10,1000,CLIENT_ID,CLIENT_SECRET,VERSION,LIMIT,500,'Utah')
slc_zc.head()


/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Venues Processed
Zip Code Analysis Complete


,PostalCode,City,Latitude,Longitude,2010 Population,Land-Sq-Mi,Density Per Sq Mile,Labels
0,84020,Draper,40.519862,-111.890216,39765,25.186,1578.853331,6
1,84047,Midvale,40.611408,-111.894511,28936,6.243,4634.951145,0
2,84065,Riverton,40.510525,-111.951605,38168,20.842,1831.302178,3
3,84070,Sandy,40.578229,-111.889815,24861,7.193,3456.276936,0
4,84081,West Jordan,40.598106,-112.031498,40732,14.627,2784.713202,2


In [32]:
slc_map=mapEscapeRooms(slc_er,slc_zc,slc_data[0],slc_data[1])
slc_map

In [60]:
slc_reco=analyzeEscapeRooms(slc_er,slc_zc)
slc_reco

,PostalCode,City,Latitude,Longitude,2010 Population,Land-Sq-Mi,Density Per Sq Mile,Labels,Number of Escape Rooms
0,84020,Draper,40.519862,-111.890216,39765,25.186,1578.853331,6,1.0
1,84047,Midvale,40.611408,-111.894511,28936,6.243,4634.951145,0,0.0
2,84070,Sandy,40.578229,-111.889815,24861,7.193,3456.276936,0,0.0
3,84088,West Jordan,40.592639,-111.972609,35603,9.419,3779.912942,0,0.0
4,84093,Sandy,40.592014,-111.834182,23130,5.423,4265.166882,0,0.0
5,84101,Salt Lake City,40.761362,-111.899539,5277,1.819,2901.044530,0,1.0
6,84102,Salt Lake City,40.759267,-111.865643,17421,1.774,9820.180383,0,0.0
7,84103,Salt Lake City,40.778311,-111.872261,21084,15.090,1397.216700,0,0.0
8,84104,Salt Lake City,40.739717,-111.914411,24869,21.233,1171.242877,0,0.0
9,84105,Salt Lake City,40.737720,-111.859591,22140,3.040,7282.894737,0,0.0


### Houston Analysis

In [39]:
tx_data=latLong('Houston, TX')
tx_er=getEscapeRooms(csv_address,CLIENT_ID,CLIENT_SECRET,VERSION,tx_data[0],tx_data[1],100000,LIMIT,'70000')
tx_er

Escape Room Data Gathering Complete
**************Recommended minimum population density to analyze:  2543.071546 *****************


,id,name,likes,dislikes,categories,city,lat,lng,formattedAddress,postalCode,2010 Population,Land-Sq-Mi,Density Per Sq Mile
0,548a4e05498eb43fd477390b,Houston Escape Room,19,0,General Entertainment,Houston,29.759877,-95.353776,"[100 Jackson St, Houston, TX 77002, United Sta...",77002,16793,2.016,8329.861111
1,54c1ba00498e215fad8c6680,Escape the Room Texas,17,0,General Entertainment,Houston,29.742100,-95.380000,"[3303 Louisiana St #220, Houston, TX 77006, Un...",77006,19664,2.255,8720.177384
2,56db53f8498e66632d7f2d82,Crazy Cat Escape Room,7,0,General Entertainment,Houston,29.787615,-95.372491,"[3231 Houston Ave, Houston, TX 77009, United S...",77009,38094,6.180,6164.077670
3,5627df71498e67243427cd00,Cipher Escape Rooms,2,0,General Entertainment,Houston,29.742755,-95.403775,"[1735 Westheimer Rd, Houston, TX 77098, United...",77098,13076,1.775,7366.760563
4,57b3d13a498ef28fab17d247,The Escape Hunt Experience - Houston,2,0,Theme Park Ride / Attraction,Houston,29.753149,-95.379632,"[125 W Gray St (W. Gray & Cushing), Houston, T...",77019,18944,3.520,5381.818182
5,5ad3b67cdd70c5492f2d8f7a,The Escape Game,2,0,General Entertainment,Houston,29.778992,-95.561390,"[794 Sorella Ct #127, Houston, TX 77024, Unite...",77024,34775,12.824,2711.712414
6,58c6a4e9d8b0ed06f9e17f39,Escape Again Rooms,1,0,Theme Park,Sugar Land,29.588139,-95.598480,"[3564 Highway 6, Sugar Land, TX 77478, United ...",77478,24739,9.728,2543.071546
7,57506450498eeb9501d9e66b,ExitusVR LLC,2,0,Theme Park Ride / Attraction,Houston,29.725261,-95.478122,"[5629 Southwest Fwy, Houston, TX 77057, United...",77057,39208,4.253,9218.904303


In [62]:
tx_zc=getZCData(csv_address,'https://www.bestplaces.net/find/zip.aspx?st=TX&city=4835000',20,500,CLIENT_ID,CLIENT_SECRET,VERSION,LIMIT,500,'Texas')
tx_zc.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Venues Processed
Zip Code Analysis Complete


,PostalCode,City,Latitude,Longitude,2010 Population,Land-Sq-Mi,Density Per Sq Mile,Labels
0,77002,Houston,29.740710,-95.381591,16793,2.016,8329.861111,19
1,77003,Houston,29.757226,-95.339080,10508,2.538,4140.267928,19
2,77004,Houston,29.628313,-95.387699,32692,5.776,5659.972299,19
3,77006,Houston,29.724805,-95.391550,19664,2.255,8720.177384,19
4,77007,Houston,29.767836,-95.361723,30853,7.809,3950.954027,19


In [63]:
tx_map=mapEscapeRooms(tx_er,tx_zc,tx_data[0],tx_data[1])
tx_map

In [64]:
tx_reco=analyzeEscapeRooms(tx_er,tx_zc)
tx_reco

,PostalCode,City,Latitude,Longitude,2010 Population,Land-Sq-Mi,Density Per Sq Mile,Labels,Number of Escape Rooms
0,77002,Houston,29.740710,-95.381591,16793,2.016,8329.861111,19,1.0
1,77003,Houston,29.757226,-95.339080,10508,2.538,4140.267928,19,0.0
2,77004,Houston,29.628313,-95.387699,32692,5.776,5659.972299,19,0.0
3,77006,Houston,29.724805,-95.391550,19664,2.255,8720.177384,19,1.0
4,77007,Houston,29.767836,-95.361723,30853,7.809,3950.954027,19,0.0
5,77008,Houston,29.799774,-95.413852,30482,6.520,4675.153374,19,0.0
6,77009,Houston,29.789127,-95.371339,38094,6.180,6164.077670,19,1.0
7,77010,Houston,29.760604,-95.370869,366,0.123,2975.609756,19,0.0
8,77011,Houston,29.733598,-95.309198,19547,3.569,5476.884281,19,0.0
9,77012,Houston,29.715451,-95.279929,20719,3.928,5274.694501,19,0.0
